In [1]:
import psycopg2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlalchemy as sa
import numpy as np
import math
import Orange
import cv2

initialize a Database, giving the username, password, server, port and the database name. Returns engine variable type

### 2.1 deteçao de mitosis com dataset

In [2]:
def initializeDb(username, password, server, port, database):
    parameters = { 
               'username': username, 
               'password': password,
               'server':   server,
                'port' : port,
               'database': database
             }
    connection = 'postgresql://{username}:{password}@{server}:{port}/{database}'.format(**parameters);
    
    return sa.create_engine(connection, encoding="utf-8")

- Adds new features to the given dataframe :
 - pctarea - change in percentage of area comparing with previous frame 
 - pctperimete - change in percentage of perimeter comparing with previous frame 
 - pctcircularity - change in percentage of the circularity comparing with previous frame 

In [ ]:
def calculateNewFeatures(features):
    features['pctarea'] = features[['id','area','areadiff','perimeter','circularity']].pct_change()['area']*100
    features['pctperimeter'] = features[['id','area','areadiff','perimeter','circularity']].pct_change()['perimeter']*100
    features['pctcircularity'] = features[['id','area','areadiff','perimeter','circularity']].pct_change(periods=1)['circularity']*100
    for index, row in features.iterrows():
        if math.isnan(row['areadiff']):
            features.loc[index,'pctarea'] = float('nan')
            features.loc[index,'pctperimeter'] = float('nan')
            features.loc[index,'pctcircularity'] = float('nan')


- Procedure that calculates the lowest or Highest change in a procedure with the previous frames
- 1st for cicle creates the needed columns names for the auxiliary dataframe
- 2nd for cicle calculates the change in percentage comparing needed frames
- 3rd for cicle set none values to incalculate values
 - frames : defines the required number of previous frames to compare with ( 5 by default )
 - minimun : true to get the lowest change, false for the highest ( true by default )
 - feature : feature selected to study
 
 

In [ ]:
def calculatePctChange(features,feature, frames=5, minimun=True):
    columns = []
    for i in range(1,frames):
        columns.append("a"+str(i+1));
    d = pd.DataFrame(columns = columns)
    for i in range(1,frames):
        d["a"+str(i+1)]= features[['id','area','areadiff','perimeter','circularity']].pct_change(periods=i+1)[feature]*100
    d['pct'+feature] = features['pct'+feature];
    for index, row in d.iterrows():
        if math.isnan(row['pct'+feature]):
            for i in range(1,frames):
                d.loc[index,"a"+str(i+1)] = float('nan')
            d.loc[index+1,"a2"] = float('nan')
        for i in range(2,frames):
            if math.isnan(row["a"+str(i)]):
                d.loc[index+1,"a"+str(i+1)] = float('nan')
    if minimun:
        features['minPct' + feature] = d.min(axis=1)
    else:
        features['maxPct' + feature] = d.max(axis=1)

Add a column to the elected dataframe that shows if the cell has values in previous frames ( 5 by default)

In [ ]:
def checkHistory(features, frames = 5):
    features['historicalRecord'] = True
    for index, row in features.iterrows():
            for i in range (0,frames):
                if index-i>=0:
                    if(features.loc[index]['frame'] - features.loc[index-i]['frame'] != i or features.loc[index]['id'] !=features.loc[index-i]['id']):
                        features.loc[index,'historicalRecord'] = False;
                        break;
                else:
                    features.loc[index,'historicalRecord'] = False;


add 2 new columns that stores if that is the last frame or the one before that one, using the cells table of that database to get the ground truth values

In [ ]:
def labelMitosis(engine,features):
    cells_table = pd.read_sql_query('select * from cells',con=engine);
    features['Mitosis'] = False;
    features['Pre-Mitosis'] = False;
    parentid_table = cells_table.groupby(['parentid']).count()
    parentid_table = parentid_table.loc[~(parentid_table.index == 0)]['id']
    tmp = parentid_table.index[:]
    for idx in tmp:
        features.loc[((features['gt_id'] == idx) & (features['frame']  == cells_table.loc[cells_table['id'] == idx].iloc[0].dead-1)),'Pre-Mitosis'] = True
        features.loc[((features['gt_id'] == idx) & (features['frame']  == cells_table.loc[cells_table['id'] == idx].iloc[0].dead)),'Mitosis'] = True

In [ ]:
def removeIncompleteCells(features):
    df = features[features.incomplete != True];
    df = df.drop(columns='incomplete');
    df = df.reset_index()
    return df;

In [ ]:
def main (username,password,server,port,database,frames=5):
   
    engine = initializeDb(username,password,server,port,database);
    
    features = pd.read_sql_query('select * from features',con=engine);
    
    features = removeIncompleteCells(features);
    
    checkHistory(features);
    
    calculateNewFeatures(features)
    
    calculatePctChange(features,'area');
    
    calculatePctChange(features,'circularity',minimun = False);
    #print features
    #labelMitosis(engine,features);
  
    return features;

In [ ]:
features = main("postgres","postgres","localhost",5434,"Fluo-N2DH-SIM+_GT")
#print features.loc(features[pre-mitosis] = True)

In [ ]:
print features.loc[features['id'] == 4]

In [ ]:
xpto = features.loc[(((features['minPctarea']>20)   | ((features['pctarea']>15) & (features['minPctarea']>5) & (features['pctcircularity']<1) & (features['maxPctcircularity'] < 3))) & (features['pctperimeter']>5) & (features['historicalRecord']==True))]
print xpto[['id','frame','pctarea','pctperimeter', 'pctcircularity','minPctarea','maxPctcircularity','Pre-Mitosis']]

## 2.2Associate cells with previous frames

In [3]:
def main2 (username,password,server,port,database):
   
    return initializeDb(username,password,server,port,database);
    
    

In [4]:
engine = main2("postgres","postgres","localhost",5434,"Fluo-N2DL-HeLa_ST")



In [25]:
def createTmpTable(engine,originalTable,tmpTable,col):
    engine.execute("drop table %s"%tmpTable)
    
    sql = "create table %s as (\
    Select * from %s)"%(tmpTable,originalTable)
    engine.execute(sql);
    
    sql = "ALTER TABLE %s \
    ADD COLUMN %s bigint ;"%(tmpTable,col)
    engine.execute(sql);
    
    sql = "ALTER TABLE %s ADD COLUMN id SERIAL PRIMARY KEY;"%(tmpTable)
    engine.execute(sql);
    
createTmpTable(engine,"lifetime","lifetmp","myId")

In [5]:
def compareCell(engine,table,frame,id):
    sql = "Select b.myid as cell1,St_area(b.geom) as area1, c.myid as cell2,c.id as idgenerated,St_area(c.geom) as area2,\
            (Select radius from ST_MinimumBoundingRadius(b.geom)),\
            ST_distance(ST_Centroid(b.geom),ST_Centroid(c.geom)) as centDistance,\
            ST_Intersects(ST_MakeValid(c.geom),ST_MakeValid(b.geom)) as l from\
            (select * from %s where id = %s ) as b,\
            (select * from %s where frame = %s) as c"%(table,id,table,frame);
    compare = pd.read_sql_query(sql ,con=engine)
    #return compare.loc[(compare['l']==True) & (compare['radius'] + 5 > compare['centdistance'])]
    return compare

In [6]:
def compareCellInFrame(engine,table,frame,id):
    sql = "Select b.myid as cell1,St_area(b.geom) as area1, c.id as cell2,c.id as idgenerated,St_area(c.geom) as area2,\
            (Select radius from ST_MinimumBoundingRadius(b.geom)),\
            ST_distance(ST_Centroid(b.geom),ST_Centroid(c.geom)) as centDistance,\
            ST_Intersects(ST_MakeValid(c.geom),ST_MakeValid(b.geom)) as l from\
            (select * from %s where frame = %s ) as b,\
            (select * from %s where id = %s) as c"%(table,frame,table,id);
    compare = pd.read_sql_query(sql ,con=engine)
    #return compare.loc[(compare['l']==True) & (compare['radius'] + 5 > compare['centdistance'])]
    return compare

In [7]:
def setInitialIDs(engine,cells_table,GT_table,table):
    cells_table = pd.read_sql_query('select * from %s'%(cells_table),con=engine)
    tmp = pd.read_sql_query('select * from %s'%(GT_table),con=engine)
    for id in cells_table.loc[cells_table['parentid'] == 0]['id']:
        try:
            firstAppear = tmp[['id','frame']].loc[tmp['id']==id].sort_values(by=['frame']).iloc[0];
            sql3 = "select * from %s \
            join %s \
            on %s.geom = %s.geom \
            where %s.frame = %s and %s.frame = %s and id = %s"%(table,GT_table,table,GT_table,GT_table,firstAppear['frame'],table,firstAppear['frame'],firstAppear['id']);
            same = pd.read_sql_query(sql3 ,con=engine)
            print same.cellid[0],same.id[0]
            engine.execute("UPDATE %s SET myId = %s WHERE id = %s;"%(table,same.cellid[0],same.id[0]));
        except:
            print "cell " + str(id) + " has no representation"

In [8]:
def restartIds(engine,table):
    sql = """ALTER TABLE %s
    drop COLUMN myId ;

    ALTER TABLE %s
    ADD COLUMN myId bigint not null default 0;"""%(table,table)
    engine.execute(sql);
restartIds(engine,"lifetmp");

In [9]:
def calculateNumberOfMitosis(engine):
    cellsGT = pd.read_sql_query('Select * from cells',con=engine);
    mitosis = cellsGT.groupby(['parentid']).count()
    return mitosis.loc[mitosis.id == 2].size/3

In [10]:
def setInitialIDs(engine,table):
    engine.execute("UPDATE %s SET myId = cellid WHERE frame = 0;"%(table));

In [11]:
def labelMitosis(engine,table,parent,child1,child2):
    if parent<100:
        engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((parent+100),child1));
        engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((parent+200),child2));
    else:
        engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((parent+10**(len(str(parent/100))+2)),child1));
        engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%((parent+2*10**(len(str(parent/100))+2)),child2));

In [12]:
def CorrespondCells(engine,table):
    lifeTmp = pd.read_sql_query('select * from %s'%(table),con=engine);
    frames = lifeTmp['frame'].nunique();
    #lastId = lifeTmp['gt_id'].nunique()
    missingVal = [];
    mitosisFrame = [];
    for i in range(1,frames):
        j = [];
        l = lifeTmp.loc[(lifeTmp['frame'] ==  i-1)];
        for index,row in l.iterrows():
            j.append(int(row.id))
        for a in j:
            compare = compareCell(engine,table,i,a)
            match = compare.loc[(compare['l']==True) & (compare['radius'] + 5 > compare['centdistance'])]
            if len(match.index) == 1:
                engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%(match.cell1.iloc[0],match.idgenerated.iloc[0]));
            elif len(match.index) == 2:
                labelMitosis(engine,table,match.cell1.iloc[0],match.idgenerated.iloc[0],match.idgenerated.iloc[1])
                
            else:
                print(a,i)
                print(len(match.index))
        
        cellsWithoutMatch = pd.read_sql_query('select * from %s where frame = %d and myid = 0'%(table,i),con=engine);
        if len(cellsWithoutMatch) > 0:
            for index1,row1 in cellsWithoutMatch.iterrows():
                closeValue = compareCellInFrame(engine,table,i-1,row1.id).sort_values("centdistance").iloc[0]
                if closeValue.centdistance < closeValue.radius*3:
                    child1 = pd.read_sql_query('select * from %s where frame = %d and myid = %d'%(table,i,closeValue.cell1),con=engine)
                    if len(child1.index) == 1:
                        labelMitosis(engine,table,closeValue.cell1,child1.iloc[0].id,row1.id);
                    elif len(child1.index) == 0:
                        engine.execute("UPDATE lifetmp SET myId = %s WHERE id = %s;"%(closeValue.cell1,row1.id));
                    else:
                        print child1
                    lifeTmp = pd.read_sql_query('select * from %s'%(table),con=engine);
            #quit();
        lifeTmp = pd.read_sql_query('select * from %s'%(table),con=engine);

In [ ]:
 elif len(match.index) == 0:
                print a
                print i
                compare = compareCellInFrame(engine,table,i,a)
                closer_values = compare.sort_values("centdistance")
                print closer_values
                #if closer_values.iloc[1].centdistance/5 < closer_values.iloc[1].area2:
                #    print("Cells %d and %d are childs of %d"%(closer_values.iloc[0]))

In [26]:
restartIds(engine,"lifetmp");
setInitialIDs(engine,"lifetmp")
CorrespondCells(engine,"lifetmp")

(82, 2)
0
(739, 3)
0
(1456, 4)
0
(2231, 5)
0
(4251, 7)
0
(5415, 8)
0
   cellid  frame                                               geom   id  myid
0       1     10  0103000000010000004F00000000000000008070400000...   90   200
1      50     10  010300000001000000550000000000000000C084400000...  136   200
(148, 12)
0
(201, 13)
0
(266, 14)
0
(316, 15)
0
(374, 15)
0
(393, 16)
0
   cellid  frame                                               geom   id  myid
0       0     17  0103000000010000005C0000000000000000A070400000...  508   200
1      59     17  0103000000010000005B0000000000000000E884400000...  563   200
(567, 18)
0
(794, 21)
0
(397, 22)
0
(806, 22)
0
(1063, 25)
0
   cellid  frame                                               geom    id  \
0       1     25  0103000000010000005E00000000000000004070400000...  1070   
1      64     25  0103000000010000005F00000000000000004885400000...  1130   

   myid  
0   200  
1   200  
(1367, 30)
0
(1474, 31)
0
(1483, 31)
0
(1544, 32)
0
   cellid 

   cellid  frame                                               geom    id  \
0      29     47  0103000000010000001F0000000000000000408B400000...  2877   
1      32     47  0103000000010000001F0000000000000000A88B400000...  2881   
2      97     47  010300000001000000520000000000000000C06C400000...  2953   

   myid  
0     0  
1     0  
2     0  
(2887, 48)
0
(2950, 48)
0
   cellid  frame                                               geom    id  \
0      28     48  010300000001000000210000000000000000408B400000...    52   
1      35     48  010300000001000000320000000000000000C881400000...  2981   
2      39     48  010300000001000000360000000000000000B880400000...  2985   
3      50     48  0103000000010000003000000000000000002087400000...  2998   
4      33     48  010300000001000000260000000000000000188C400000...  2979   
5      97     48  010300000001000000530000000000000000606D400000...  3049   

   myid  
0     0  
1     0  
2     0  
3     0  
4     0  
5     0  
   cellid  fram

   cellid  frame                                               geom    id  \
0      48     76  0103000000010000003700000000000000008045400000...  4775   
1      45     76  010300000001000000330000000000000000005C400000...  6108   
2      54     76  0103000000010000002400000000000000001079400000...  6163   
3      59     76  0103000000010000002A0000000000000000907F400000...  6168   
4      75     76  0103000000010000003600000000000000001072400000...  6186   

   myid  
0     0  
1     0  
2     0  
3     0  
4     0  
(6100, 77)
0
(6163, 77)
0
(6168, 77)
0
   cellid  frame                                               geom    id  \
0      47     77  0103000000010000003700000000000000000043400000...  6270   
1      46     77  010300000001000000340000000000000000005C400000...  6269   
2      64     77  0103000000010000003B0000000000000000407F400000...  6289   

   myid  
0     0  
1     0  
2     0  
   cellid  frame                                               geom    id  \
0      47   

In [27]:
def programCycle(engine,table):
    sql = '''select myid, inicio, fim,
    (select DISTINCT b.myid iniciopai
    from %s b
    where b.myid = MOD(a.myid,100)) iniciopai
    from
    (select myid, min(frame) inicio, max(frame) fim
    from lifetmp
    group by myid) a
    order by iniciopai asc, inicio desc'''%(table)

    pgCycle = pd.read_sql_query(sql,con=engine);
    #print pgCycle
    return pgCycle


In [28]:
def getProgramLifeCycle(engine,table):
    cycle = programCycle(engine,table)
    tmp = cycle.groupby(['iniciopai']).apply(lambda x: x['fim'].values)
    df = tmp.to_frame()  
    df2 = pd.DataFrame( columns=['id2','deaths']);
    columns = list(df2)
    data2 = [];
    for index, row in df.iterrows():
        a = row[0].tolist()
        a.append(cycle.loc[cycle.myid==row.name].inicio.iloc[0])
        a.sort()
        if index > 0:
            values = [index,[int(v) for v in a ]];
            zipped = zip(columns, values)
            a_dictionary = dict(zipped)
            data2.append(a_dictionary)

    df2 = df2.append(data2,True)
    return df2
lifeCycle = getProgramLifeCycle(engine,"lifetmp")

In [29]:
def getAllDeathFrames(id,table,arr):
    arr.append(id)
    sons = table.loc[table['parentid']==id].id.unique();
    if sons.size > 0:
        for a in sons:
            getAllDeathFrames(a,table,arr)
    else:
        return arr
    return arr

In [30]:
def getGTLifeCycle(engine,cellsTable):
    cellsGT = pd.read_sql_query('select * from %s'%(cellsTable),con=engine)
    df = pd.DataFrame( columns=['id','deaths']);
    columns = list(df)
    data = [];
    initCells = cellsGT.loc[cellsGT['parentid']==0].id.unique();

    for cell in initCells:
        family = getAllDeathFrames(cell,cellsGT,[]);
        deaths = cellsGT.loc[cellsGT['id'].isin(family)]['dead'];
        a = deaths.values.tolist();
        a.append(cellsGT.loc[cellsGT['id'] == cell]['born'].iloc[0])
        a.sort();
        values = [cell,[int(v) for v in a ]];
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)
    df = df.append(data,True)
    return df
GTCycle = getGTLifeCycle(engine,"cells")
#print GTCycle.iloc[7].deaths

In [31]:
def compareValues(dataframe1,dataframe2):
    data = [];
    df = pd.DataFrame( columns=['id','id2','deaths']);
    columns = list(df);
    for index,row in dataframe1.iterrows():
        for index2,row2 in dataframe2.iterrows():
            try:
            #(np.isin(np.array(dataframe1['deaths'][index]),np.array(dataframe2['deaths'][index2]))).all() and len(np.array(dataframe1['deaths'][index]))    
                test1 = np.array(dataframe1['deaths'][index])[:2];
                test2 = np.array(dataframe2['deaths'][index2])[:2];
                if (test1 == test2).all():
                    values = [dataframe1['id'][index],dataframe2['id2'][index2],test1]
                    zipped = zip(columns, values)
                    data_dictionary = dict(zipped)
                    data.append(data_dictionary)
            except:
                continue
    
    df = df.append(data,True)
    return df
print lifeCycle
print GTCycle
df = compareValues(GTCycle,lifeCycle)        
print df

   id2                                             deaths
0    1                                     [0, 6, 14, 15]
1    2  [0, 7, 18, 20, 34, 39, 41, 70, 74, 74, 75, 76,...
2    3  [0, 3, 8, 9, 41, 44, 46, 46, 52, 53, 56, 64, 8...
3    4                            [0, 20, 55, 58, 91, 91]
4    5  [0, 5, 40, 41, 58, 59, 69, 70, 73, 74, 75, 81,...
5    6                    [0, 12, 23, 42, 85, 91, 91, 91]
6    7  [0, 13, 23, 26, 27, 30, 31, 32, 33, 45, 49, 52...
7    8  [0, 2, 37, 37, 47, 49, 50, 51, 53, 68, 70, 72,...
8    9            [0, 25, 30, 34, 35, 40, 47, 55, 56, 56]
9   10                            [0, 13, 21, 22, 30, 31]
10  11                                     [0, 9, 11, 29]
11  12                                            [0, 13]
12  13                                             [0, 6]
13  14  [0, 8, 13, 14, 43, 48, 49, 50, 51, 65, 80, 81,...
14  15  [0, 14, 38, 39, 40, 42, 44, 46, 57, 70, 71, 72...
15  16                            [0, 35, 36, 36, 91, 91]
16  17        

In [32]:
def evaluateGTMitosis(dataframe):
    mitosis = 0
    for index,row in dataframe.iterrows():
        if row.deaths[0] == 0:
            mitosis += len(row.deaths)/2-1
    print "There are %s mitosis from initial cells"%(mitosis)
    return mitosis
evaluateGTMitosis(GTCycle)

There are 92 mitosis from initial cells


92

In [33]:
def evaluateDetectedMitosis(dataframe):
    mitosis = 0
    for index,row in dataframe.iterrows():
        if row.deaths[0] == 0:            
            mitosis += len(row.deaths)/2-1
    print "There are %s mitosis well detected"%(mitosis)
    return mitosis
evaluateDetectedMitosis(df)

There are 0 mitosis well detected


0

In [173]:
def getFirstGenId(id,cellsTable):
    parent = cellsTable.loc[cellsTable.id==id].iloc[0].parentid;
    if(parent==0):
        return id
    else:
        return getFirstGenId(parent,cellsTable)

In [254]:
def createGTMitosisDf(engine,cellsTable):
    cellsGT = pd.read_sql_query('select * from %s'%(cellsTable),con=engine)
    mitosisdf = pd.DataFrame( columns=['firstGen','born','dead']);
    #apoptosisdf = pd.DataFrame( columns=['firtsGen','born','dead'])
    columns = list(mitosisdf)
    data1 = [];
    #data2 = [];

    for index,row in cellsGT.iterrows():
        values = [getFirstGenId(row.id,cellsGT),row.born,row.dead];
        zipped = zip(columns,values)
        a_dictionary = dict(zipped)

        tmp = cellsGT.loc[cellsGT['parentid'] == row.id]
        if len(tmp.index) == 0:
            continue;
            #print "cell %d has no mitosis"%row.id
            #data2.append(a_dictionary)
        else:
            data1.append(a_dictionary)
            #print "cell %d is parent from "%row.id,tmp.iloc[0].id,tmp.iloc[1].id
            
    mitosisdf = mitosisdf.append(data1,True)
    #apoptosisdf = apoptosisdf.append(data2,True)
    return mitosisdf

In [255]:
def createProgramMitosisDf(engine,table):
    cycle = programCycle(engine,table)
    mitosisdf = pd.DataFrame( columns=['firstGen','born','dead']);
    #apoptosisdf = pd.DataFrame( columns=['firstGen','born','dead'])
    columns = list(mitosisdf)
    data1 = [];
    #data2 = [];
    for index,row in cycle.iterrows():
        values = [row.myid%100,row.inicio,row.fim];
        zipped = zip(columns,values)
        a_dictionary = dict(zipped)
        if row.myid>100: 
            val = 10**(len(str(row.myid/100))+2)
        else:
            val = 100
        tmp = cycle.loc[(((cycle['myid']/val) > 1) & (cycle['myid']%val==row.myid))]['myid']
        if len(tmp.index) > 0:
            data1.append(a_dictionary)
    mitosisdf = mitosisdf.append(data1,True)
    return mitosisdf

In [256]:
def correspondIds(engine,programDf,GtDb,port,user,pw):
    
    sql = "Select cellid, a.geom, a.gtId from lifetime, \
    (SELECT geoms.* \
    FROM dblink('dbname=%s port=%s user=%s password=%s', \
    'SELECT * FROM lifetime where frame = 0') \
    AS geoms(gtId smallint ,frame smallint , geom geometry)) as a \
    where ST_Contains(lifetime.geom,ST_centroid(a.geom)) and lifetime.frame = 0 order by cellid"%(GtDb,port,user,pw)

    tmp = pd.read_sql_query(sql,con=engine)
    for index,row in tmp.iterrows():
        programDf.loc[programDf['firstGen'] == row.cellid, 'gtId'] = row.gtid

In [257]:
def totalMitosisDetected(gtDf,programDf):
    tmp = gtDf.groupby(['born','dead']).sum().reset_index()
    wellDetected = 0
    #possibleWellDetected = 0
    for index,row in tmp.iterrows():
        a = gtDf.loc[((gtDf['born']==row.born) & (gtDf['dead']==row.dead))]
        b = programDf.loc[((programDf['born']==row.born) & (programDf['dead']==row.dead))]
        
        
        if len(b)>0:
            for index1,row1 in b.iterrows():
                if(len(a.loc[(a['firstGen']==row1.gtId) ])>0):
                    wellDetected += 1;
            #possibleWellDetected += min(len(b.loc[b['gtId']==0]),len(a))
    return wellDetected
    #print possibleWellDetected  

In [258]:
def totalMitosisFromInitialCells(gtDf,programDf):
    count = 0;
    for index,row in gtDf.iterrows():
        if len(programDf.loc[programDf['gtId'] == row.firstGen])>0:
            count+=1;
    return count
totalMitosisFromInitialCells(mitosisdf,mitosisdf2)

75

In [259]:
def totalFalsePositives(gtDf,programDf):
    return len(programDf.loc[programDf['firstGen'] != 0])- totalMitosisDetected(gtDf,programDf)

In [262]:
def evaluateResults(engine):
    gtDf = createGTMitosisDf(engine,"cells");
    programDf = createProgramMitosisDf(engine,"lifetmp");
    correspondIds(engine,programDf,"Fluo-N2DL-HeLa",5434,"postgres","postgres")
    accuratedMitosis = totalMitosisDetected(gtDf,programDf);
    totalMitosis = totalMitosisFromInitialCells(gtDf,programDf);
    print "There are %s out of %s mitosis well detected meaning %s percent . There are %s falsepositives"%(accuratedMitosis,totalMitosis,float(accuratedMitosis)/totalMitosis*100,totalFalsePositives(gtDf,programDf))

    
evaluateResults(engine)

There are 54 out of 75 mitosis well detected meaning 72.0 percent . There are 140 falsepositives


In [172]:
programDf

NameError: name 'programDf' is not defined